In [ ]:
import os
import simplejson as json

class Prepare:
    def __init__(self):
        self.json_name = '1.json'
        with open(self.json_name) as data_file:    
            self.json = json.load(data_file)
        self.path = self.json['path0']
    
    # add all scenes from landsats_done
    def add_scenes_to_json(self):
        dirs = [d for d in os.listdir(self.path) if os.path.isdir(os.path.join(self.path, d))]
        self.json['scenes'] = []
        id = 0
        for d in dirs:
            tmp = {}
            tmp['id'] = id
            id += 1
            tmp['path'] = d
            tmp['bit'] = '16' if (d[2] == '8') else '8'
            self.json['scenes'].append(tmp)
        with open(self.json_name, 'w') as outfile:
            json.dump(self.json, outfile, sort_keys = True, indent = 4, ensure_ascii = False)

    # copy all need files from landsats_done
    def copy_need_files(self):
        path_in = self.json['path']
        dirs = [os.path.join(path_in, i) for i in 
                ['in', 'in_filtered', 'in/red', 'in/nir', 'in_filtered/red', 'in_filtered/nir']]
        try:
            for i in dirs:
                os.mkdir(i)
        except:
            raise Exception('copy_need_files: dirs can not be created or exist')
        for i in self.json['scenes']:
            if (i['bit'] == '8'):
                suffs = ['_B3_C.tif', '_B4_C.tif', '_B3_C_F.tif', '_B4_C_F.tif']
            else:
                suffs = ['_B4_C.tif', '_B5_C.tif', '_B4_C_F.tif', '_B5_C_F.tif']
            for j in range(len(suffs)):
                os.system("cp " + os.path.join(self.path, i['path'], i['path'] + suffs[j]) + " " + 
                          os.path.join(dirs[j + 2], i['path'] + '.tif'))
    
#pr = Prepare()
#pr.add_scenes_to_json()
#pr.copy_need_files()

In [4]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [39]:
%%cython
import os
import simplejson as json
import numpy as np
cimport numpy as np
cimport cython
import skimage.io
import matplotlib
#from libc.math cimport abs

# class for one scene analyse
cdef class Scene:
    cdef public int id, bit
    cdef object json, path
    def __init__(self, _id_):
        self.id = _id_
        with open('1.json') as data_file:    
            self.json = json.load(data_file)
        self.bit = int(self.json['scenes'][self.id]['bit'])
        self.path = self.json['scenes'][self.id]['path']
    
    def read_images(self, path):
        return {'red': skimage.io.imread(os.path.join(self.json['path'], self.json['paths'][path], 'red',
                                                     self.path +  self.json['suffs'][path])),
                'nir': skimage.io.imread(os.path.join(self.json['path'], self.json['paths'][path], 'nir',
                                                     self.path +  self.json['suffs'][path]))}
    
    cdef is_in_mask(self, unsigned short val):
        return val < 64000 if self.bit == 16 else val < 253
    
    def get_plot_array(self, imgs):
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        sizes = self.json['plotSizes'][str(self.bit)]
        cdef int sn0 = sizes['nir'][0], sn1 = sizes['nir'][1], sr0 = sizes['red'][0], sr1 = sizes['red'][1]
        cdef np.ndarray[np.int32_t, ndim = 2] plot_array = np.zeros([sr1 - sr0 + 1, sn1 - sn0 + 1], dtype = 'int32')
        cdef int x, y
        for x in range(img_red.shape[1]):
            for y in range(img_red.shape[0]):
                if self.is_in_mask(img_red[y, x]) and self.is_in_mask(img_nir[y, x]) \
                    and sr0 <= img_red[y, x] <= sr1 and sn0 <= img_nir[y, x] <= sn1:
                        plot_array[img_nir[y, x] - sn0, img_red[y, x] - sr0] += 1
        return plot_array
    
    def save_plot_array(self, plot_array):
        matplotlib.image.imsave(os.path.join(self.json['path'], self.json['paths']['notMarkedPlots'], 
                                             self.path + self.json['suffs']['markedPlots']), 
                                plot_array[::-1, :])
    def make_solp_scene(self):
        imgs = self.read_images('inFiltered')
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        sizes = self.json['plotSizes'][str(self.bit)]
        cdef int sn0 = sizes['nir'][0], sn1 = sizes['nir'][1], sr0 = sizes['red'][0], sr1 = sizes['red'][1]
        cdef np.ndarray[np.uint8_t, ndim = 3] plot_marked = skimage.io.imread(
            os.path.join(self.json['path'], self.json['paths']['markedPlots'], 
                         self.path + self.json['suffs']['markedPlots']))
        cdef np.ndarray[np.uint8_t, ndim = 2] colors = \
            np.array(self.json['colorsMarkedPlots']).astype('uint8', copy = False)
        cdef int x, y, need_white, red, nir, color_eps = 9, color_sum, c, cs
        cdef int __here__ = 0
        for x in range(img_red.shape[1]):
            for y in range(img_red.shape[0]):
                red, nir = img_red[y, x], img_nir[y, x]
                need_white = 0
                if not sr0 <= red <= sr1 or not sn0 <= nir <= sn1:
                    need_white = 1
                else:
                    for c in range(colors.shape[0]):
                        color_sum = 0
                        for cs in range(3):
                            color_sum += abs(colors[c, cs] - plot_marked[sn1 - nir, red - sr0, cs])
                        if (color_sum > color_eps):
                            need_white = 0
                            break
                if need_white == 1:
                    __here__ += 1
                    img_nir[y, x] = img_red[y, x] = 65535 if self.bit == 16 else 255
        for ch in ['red', 'nir']:
            skimage.io.imsave(os.path.join(self.json['path'], self.json['paths']['solp'], ch, 
                                           self.path + self.json['suffs']['solp']), 
                              imgs[ch], plugin = 'tifffile')
        print __here__
    
    def get_soil_line(self, imgs):
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 1] red = np.zeros([img_red.size], dtype = 'uint16')
        cdef np.ndarray[np.uint16_t, ndim = 1] nir = np.zeros([img_red.size], dtype = 'uint16')
        cdef int x, y, size = 0
        for x in range(img_red.shape[1]):
            for y in range(img_red.shape[0]):
                red[size] = img_red[y, x]
                nir[size] = img_nir[y, x]
                size += 1
        np.resize(red, size)
        np.resize(nir, size)
        return np.linalg.lstsq(np.vstack([red, np.ones(len(red))]).T, nir)[0]
                    
    cdef brat_vec(self, np.ndarray[np.uint16_t, ndim = 2] img_red, np.ndarray[np.uint16_t, ndim = 2] img_nir,
        int pnt_x, int pnt_y, int rad, float line_a, float line_b):
        cdef float res_x = 0, res_y = 0, ave_x = 0, ave_y = 0, ave_subj = 0
        cdef int n = 0, x, y
        for y in xrange(max(pnt_y - rad, 0), min(pnt_y + rad, img_red.shape[0])):
            for x in xrange(max(pnt_x - rad, 0), min(pnt_x + rad, img_red.shape[1])):
                if (pnt_x - x) ** 2 + (pnt_y - y) ** 2 <= rad ** 2 \
                    and self.is_in_mask(img_red[y, x]) and self.is_in_mask(img_nir[y, x]):
                        n += 1
                        subj = float(-line_a) * img_red[y, x] - line_b + img_nir[y, x]
                        res_x += x * subj
                        res_y += y * subj
                        ave_x += x
                        ave_y += y
                        ave_subj += subj
        return {'xx': (res_x - ave_x * ave_subj) / float(n) if n > 0 else -9999,
               'yy': (res_y - ave_y * ave_subj) / float(n) if n > 0 else -9999,
               'avg': ave_subj / float(n) if n > 0 else -9999,
               'n': n}
    
    def brat_first(self):
        json_f1 = self.json['brat_first']
        imgs = self.read_images('in')
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        cdef int rad = json_f1['rad'], x_pts = json_f1['xPts'], y_pts = json_f1['yPts']
        cdef int line_a, line_b
        line_a, line_b = self.get_soil_line(imgs)
        cdef int x_abs, y_abs, i, j, x, y, x_abs_0 = json_f1['xAbs0'], y_abs_0 = json_f1['yAbs0']
        cdef int x_abs_step = json_f1['xAbsStep'],  y_abs_step = json_f1['yAbsStep']
        cdef int need_comma = 0
        
        f = open(os.path.join(self.json['path'], self.json['paths']['bratTxt'], self.path + \
                              self.json['suffs']['bratTxt']), 'w')
        for i in range(x_pts):
            for j in range(y_pts):
                x = int(i / (x_pts - 1.0) * img_red.shape[1])
                y = int(j / (y_pts - 1.0) * img_red.shape[0])
                x_abs = x_abs_0 + x * x_abs_step
                y_abs = y_abs_0 + y * y_abs_step
                vec = self.brat_vec(img_red, img_nir, x, y, rad, line_a, line_b)
                if need_comma:
                    f.write(',')
                f.write('{{%i,%i},{%i,%i},{%f,%f},%f,%i}\n' % 
                        (x, y, x_abs, y_abs, float(vec['xx']), float(vec['yy']), float(vec['avg']), vec['n']))
                if not need_comma:
                    need_comma = 1
        f.write('}')
        f.close()
        
        
        
    

In [45]:
A = Scene(0)
#imgs = A.read_images('inFiltered')
#plot_array = A.get_plot_array(imgs)
#A.save_plot_array(plot_array)
#A.brat_f1()
A.make_solp_scene()
#A.brat_first()
#imgs = A.read_images('in')

CPU times: user 191 ms, sys: 16.1 ms, total: 207 ms
Wall time: 207 ms
